# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [31]:
import logging
import os
import csv
import joblib 

from matplotlib import pyplot as plt
import numpy as np
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
import pandas as pd
from sklearn import datasets
import pkg_resources
from azureml.core import Dataset
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.webservice import LocalWebservice


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset that I am using is a binary classification dataset using wine chemical data. The dataset attempts to classify wine as either 'good' or 'bad' according to professional tasters. The input data contains various data related to consistency and taste parameters of wine. The following AutoML run will train an array of models and rank them based upon their accuracy scores

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
def preprocess(data):
    y_df = data.pop("quality").apply(lambda x: 1 if x == "good" else 0)
    return data, y_df


ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='Wine')
dataset = dataset.to_pandas_dataframe()
x, y = preprocess(dataset)
x['y'] = y
os.makedirs('data', exist_ok=True)
x.to_parquet('data/wine.parquet')
dataref = ws.get_default_datastore().upload('data')
dataset = Dataset.Tabular.from_parquet_files(path = dataref.path('wine.parquet'))

# choose a name for experiment
experiment_name = 'auto-ml-exp-1'

experiment=Experiment(ws, experiment_name)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading data/wine.parquet
Uploaded data/wine.parquet, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'accuracy'
}


# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "classification",
    training_data=dataset,
    label_column_name="y",   
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml-exp-1,AutoML_8f3d37e0-5b14-4d47-bd94-51c60d81aeff,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
overall_best_run, best_model = remote_run.get_output()
best_run_metrics = overall_best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-core, training version:1.38.0, current version:1.37.0
Package:azureml-dataprep, training version:2.26.0, current version:2.25.0
Package:azureml-dataprep-rslex, training version:2.2.0, current version:2.1.0
Package:azureml-dataset-runtime, training version:1.38.0, current version:1.37.0
Package:azureml-defaults, training version:1.38.0, current version:1.37.0
Package:azureml-interpret, training version:1.38.0, current version:1.37.0
Package:azureml-mlflow, training version:1.38.0, current version:1.37.0
Package:azureml-pipeline-core, training version:1.38.0, current version:1.37.0
Package:azureml-responsibleai, training version:1.38.0, current version:1.37.0
Package:azureml-telemetry, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-client, training version:1.38.0, current version:1.37.0
Package:azureml-train-automl-runtime, training version:1.38.0, current version

accuracy 0.8248905565978736
norm_macro_recall 0.6487360413837402
recall_score_micro 0.8248905565978736
f1_score_micro 0.8248905565978736
AUC_macro 0.8870019503416584
recall_score_weighted 0.8248905565978736
AUC_weighted 0.8870019503416584
recall_score_macro 0.8243680206918701
average_precision_score_weighted 0.8826075974427455
AUC_micro 0.8871316148577852
f1_score_macro 0.8238826573028587
matthews_correlation 0.6482806478694637
average_precision_score_micro 0.8833567234347965
weighted_accuracy 0.8254050291305587
precision_score_micro 0.8248905565978736
f1_score_weighted 0.8249368211643594
average_precision_score_macro 0.881092301767476
precision_score_macro 0.8239149768568707
log_loss 0.4256158806388208
balanced_accuracy 0.8243680206918701
precision_score_weighted 0.8254955955116717
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_8f3d37e0-5b14-4d47-bd94-51c60d81aeff_45/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_8f3d37e0-5b14-4d47-bd94-51c60d81a

In [16]:
#TODO: Save the best model
os.makedirs('output', exist_ok=True)
joblib.dump(best_model, filename='./output/automl_model.pkl')

['./output/automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:
import os
if "source_dir" not in os.listdir():
    os.mkdir("./source_dir")

In [50]:
%%writefile score.py
import json
import os 
import joblib

def init():
    global model
    # Replace filename if needed.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automl_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)


def run(data):
    test = json.loads(data)
    print(test)
    return model.predict(test)

Overwriting score.py


In [51]:
!cp score.py ./source_dir

In [52]:
# register model
best_run = remote_run.get_best_child()
print(best_model.steps)
model_name = best_run.properties['model_name']
description = 'Wine Classification Model - AutomML'
model = remote_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = None)

print(model.name, model.id, model.version, sep='\t')


env = Environment(name="project_environment")
python_packages = ['joblib', 'numpy']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./score.py",
)



deployment_config = LocalWebservice.deploy_configuration(port=6789)
# deployment_config = AciWebservice.deploy_configuration(
#     cpu_cores=0.5, memory_gb=1, auth_enabled=False
# )
service = Model.deploy(
    ws,
    "wine-prediction",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=False,
    enable_feature_sweeping=True,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('31', Pipeline(
        memory=None,
        steps=[('standardscalerwrapper', StandardScalerWrapper(
            copy=True,
            with_mean=False,
            with_std=False
        )), ('xgboostclassifier', XGBoostClassifier(
            random_state=0,
            n_jobs=1,
            problem_info=ProblemInfo(
                gpu_training_param_dict={'processing_unit_type': 'cpu'}
            ),
            booster='gbtree',
            colsample_bytree=0.7,
            eta=0.01,
            gamma=0,
            max_depth=8,
            max_leaves=0,
            n_estimators=100,
            o

ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [21]:
import requests
import json
uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "fixed acidity":7.3,	
    "volatile acidity": 0.9,	
    "citric acid": 0.1,	
    "residual sugar": 1.9,	
    "chlorides": 0.076,	
    "free sulfur dioxide": 11.0, 	
    "total sulfur dioxide": 34.0,	
    "density": 0.9978,	
    "pH": 3.49,	
    "sulphates": 0.50,	
    "alcohol": 9.4
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
response.text

'Unauthorized, no token matched'

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
